In [77]:
import pandas as pd
import pandas as pd
import pandas as pd
from complexcgr import CGR
from pathlib import Path
# Instantiate class CGR
cgr = CGR()

def get_last_name(file_path):
    return Path(file_path).stem

def preprocessing(snp_list,df_bio,df_sra,country):
    # Step1: merge SNPs of all samples
    with open(snp_list, "r") as f:
        sample_files = f.read().splitlines()

    try:
        # Read the data from the first sample
        first_data = pd.read_csv(sample_files[0], sep=" ", header=None)
        first_data.columns = ["genomic_position", "REF", get_last_name(sample_files[0])]
        # Drop the REF column
        first_data = first_data.drop(columns=["REF"])

        # Start with first_data as the merged data
        merge_data = first_data

        # add other samples and merge them one by one
        for sample in sample_files[1:]:
            try:
                pd.read_csv(sample, sep=" ", header=None)
#                 print(sample)
                sample_data = pd.read_csv(sample, sep=" ", header=None)
                column_name = get_last_name(sample)
                sample_data.columns = ["genomic_position", "REF", column_name]
                # Drop the REF column
                sample_data = sample_data.drop(columns=["REF"])
                # merge data on genomic positions
                merge_data = merge_data.merge(sample_data, on="genomic_position", how="outer")
            except:
                continue

        merge_data.set_index("genomic_position", inplace=True)
        # Tranpose the df (reverse) to have samples as rows and genomic positions as columns
        merge_data = merge_data.T
        #replace nan with merged data
#        df=df.merge_data("N")
        df=merge_data.fillna("N")
        
        df=df.reset_index()
        df=df.rename(columns={"index":"sra_accession"})
        
        # Step 2: Remove constant columns
        non_constant_cols = df.columns[df.nunique() > 1]
        df_constant_dropped = df[non_constant_cols]
        df_constant_dropped
        df_constant_dropped.insert(1, "country", country)
        # read metadata
        df_bio=pd.read_table(df_bio)
        df_sra=pd.read_table(df_sra,header=None)
        #map sra to biosample
        df_sra_biosample=pd.concat([df_bio,df_sra], axis=1)
         # concat the sra and df
        df_sra_biosample.dropna(inplace=True)
        df_country=pd.concat([df_sra_biosample,df],axis=1)
        df_country
        
        df_constant_dropped.to_csv(f"../data/{country}_processed.csv")

        # Move the return statement outside the loop, after all samples have been processed and merged.
        return df_constant_dropped

    except Exception as e:
        print("Error:", str(e))
        return None


# Call the preprocessing function and store the result in the 'df' variable
#     merge_data = merge_data.fillna("N")
#     merge_data = merge_data.replace({"A": 1, "G": 2, "C": 3, "T": 4, "N": 0})
#     merge_data = merge_data.rename(columns={"V1": "POS"})
#     tinput = merge_data.T
#     tinput.columns = tinput.iloc[0]
#     tinput = tinput.iloc[1:]
#     value = len(sample_files) // 5 * 4
#     tinput4 = tinput[tinput["Num"] < value]
#     tinput5 = tinput4.T
#     tinput5.to_csv("snp_input.csv", index=False)
#     # FCGR Encoding using kaos package
#     data = pd.read_csv("snp_input.csv")
#     data = data.replace({0: "N", 1: "A", 2: "G", 3: "C", 4: "T"})

#     for i in range(900):
#         cgr_list = cgr(data[str(i)], seq_base=["N", "A", "G", "C", "T"], res=200)
#         cgr_matrix = cgr_list["matrix"]
#         cgr_matrix.to_csv(sample_files[i], index=False)


In [82]:
df_kenya = preprocessing(snp_list="../data/snp.lists",
                         df_bio="../accessions/kenya/kenya.tsv",
                         df_sra="../pipeline/run-2023-08-11-salmonellaml/kenya_biosamples.txt",
                         country="kenya")

In [80]:
df_tz = preprocessing(snp_list="../data/snp.lists",
                      df_bio="../work/5a/c62ab92992b24ec6c72c7c94c7903d/tanzania_biosamples.txt",
                      df_sra="../pipeline/run-2023-08-11-salmonellaml/sra_tanzania_biosamples.txt",
                      country="tanzania")

In [81]:
df_tz

genomic_position,sra_accession,country,22,23,36,43,216,480,495,534,...,4841673,4841793,4844655,4851050,4854011,4854114,4854186,4854195,4854340,4855889
0,SRR11005771,tanzania,G,C,G,C,T,G,C,C,...,N,N,N,N,N,N,N,N,N,N
1,SRR11005772,tanzania,G,C,G,C,T,G,C,C,...,N,N,N,N,N,N,N,N,N,N
2,SRR11005773,tanzania,G,C,G,C,N,G,C,C,...,N,N,N,N,N,N,N,N,N,N
3,SRR11005774,tanzania,N,N,N,N,N,G,N,C,...,N,N,N,N,N,N,N,N,N,N
4,SRR11005775,tanzania,N,N,N,N,N,G,N,C,...,N,N,N,N,N,N,N,N,N,N
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
71,SRR8980095,tanzania,N,N,N,N,N,G,N,C,...,N,N,N,N,N,N,N,N,N,N
72,SRR8980096,tanzania,N,N,N,N,N,G,N,C,...,N,N,N,N,N,N,N,N,N,N
73,SRR8980097,tanzania,G,C,N,N,N,G,N,C,...,N,N,N,N,N,N,N,N,N,N
74,SRR8980099,tanzania,N,N,N,N,N,G,N,C,...,N,N,N,N,N,N,N,N,N,N


In [90]:
# inspect the constant columns
# Step 2: Remove constant columns
def non_constant(df):
    constant_cols = df.columns[df.nunique() ==1]
#     constant = df[constant_cols]
    constant
    return constant

# Non constant Regions

In [91]:
non_constant(df_tz)

genomic_position,136569,185378,250051,277015,286832,410830,441536,459590,549075
0,G,G,C,C,A,C,T,C,C
1,G,G,C,C,A,C,T,C,C
2,G,G,C,C,A,C,T,C,C
3,G,G,C,C,A,C,T,C,C
4,G,G,C,C,A,C,T,C,C
...,...,...,...,...,...,...,...,...,...
127,G,G,C,C,A,C,T,C,C
128,G,G,C,C,A,C,T,C,C
129,G,G,C,C,A,C,T,C,C
130,G,G,C,C,A,C,T,C,C


In [89]:
non_constant(df_kenya)

genomic_position,country
0,kenya
1,kenya
2,kenya
3,kenya
4,kenya
...,...
71,kenya
72,kenya
73,kenya
74,kenya


# Nigeria


In [35]:
# Nigeria
# load data
df_nigeria=pd.read_table("../data/nigeria.tsv")
df_nigeria.head()

,#Organism group,Strain,Isolate identifiers,Serovar,Isolate,Create date,Location,Isolation source,Isolation type,SNP cluster,Min-same,Min-diff,BioSample,Assembly,AMR genotypes,Computed types
0,Salmonella enterica,RKS304_601-84,"BCW_2606,""CFSAN005617"",""RKS304_601-84"",""SRS536...","1,40:z35:-",PDT000002955.5,2020-04-27T12:13:59Z,Nigeria,food,environmental/other,NaN,NaN,NaN,SAMN02368165,NaN,mdsB=COMPLETE,"antigen_formula=40:z35:-,serotype=[1],40:z35:-"
1,Salmonella enterica,33(3),"33(3),""BCW_2809"",""SRS538850""",Uganda,PDT000003053.4,2015-12-12T15:21:45Z,Nigeria,"Human, faeces",clinical,PDS000001850.5,NaN,0.0,SAMN02368677,GCA_010639825.1,"mdsA=COMPLETE,mdsB=COMPLETE","antigen_formula=3,10:l,z13:1,5,serotype=Uganda"
2,Salmonella enterica,FMA0095,"FMA0095,""SRS565883""",Ealing,PDT000003898.3,2015-02-11T02:07:39Z,Nigeria,melon seed,environmental/other,NaN,NaN,NaN,SAMN02345045,GCA_009997545.1,"mdsA=COMPLETE,mdsB=COMPLETE","antigen_formula=35:g,m,s:-,serotype=Ealing"
3,Salmonella enterica,BCW_2640,"9(3),""BCW_2640"",""SRS533427""",Ank,PDT000047946.4,2015-07-23T05:10:39Z,Nigeria,Cattle liver,environmental/other,NaN,NaN,NaN,SAMN02368530,GCA_010369405.1,"mdsA=COMPLETE,mdsB=COMPLETE","antigen_formula=28:k:e,n,z15,serotype=Ank"
4,Salmonella enterica,FDA904406 1-1,"CFSAN032276,""FDA00008919"",""FDA904406 1-1"",""SRS...",Pomona,PDT000060513.2,2015-04-18T14:10:49Z,Nigeria,ground chili pepper,environmental/other,PDS000031411.1,1.0,NaN,SAMN03483118,GCA_011409935.1,"mdsA=COMPLETE,mdsB=COMPLETE","antigen_formula=28:y:1,7,serotype=Pomona"


In [60]:
import pandas as pd

# Load the data
# Filter the DataFrame
filtered_df = df_nigeria[(df_nigeria['Isolation source'].notnull()) & (~df_nigeria['Isolation source'].isin(['feces', 'stool']))]
 # Assuming df_nigeria is your original DataFrame
filtered_poultry = df_nigeria[df_nigeria['Isolation source'].isin(['poultry feed','Poultry vendoors and farms','chicken heart','Poultry small intenstine','Poultry carcass','chicken liver','Chicken Spleen','chicken','poultry water'])]                             
# Display the filtered DataFrame
filtered_poultry.to_csv("../accessions/nigeria/filtered_nigeria_nigeria_poultry_data.tsv", sep='\t', index=False)
filtered_poultry

,#Organism group,Strain,Isolate identifiers,Serovar,Isolate,Create date,Location,Isolation source,Isolation type,SNP cluster,Min-same,Min-diff,BioSample,Assembly,AMR genotypes,Computed types
88,Salmonella enterica,BCW_2822,"66(3),""BCW_2822"",""SRS528782""",Wilhelmsburg,PDT000197317.1,2017-03-22T14:56:21Z,Nigeria,Poultry small intenstine,environmental/other,NaN,NaN,NaN,SAMN02368689,GCA_002062955.1,"mdsA=COMPLETE,mdsB=COMPLETE","antigen_formula=4:z38:-,serotype=Wilhelmsburg"
89,Salmonella enterica,BCW_2817,"BCW_2817,""KR 16"",""SRS865334""",Virchow,PDT000197326.1,2017-03-22T15:02:25Z,Nigeria,chicken,environmental/other,PDS000031736.79,22.0,16.0,SAMN02368684,GCA_002063155.1,"dfrA5=COMPLETE,gyrA_S83Y=POINT,mdsA=COMPLETE,m...","antigen_formula=7:r:1,2,serotype=Virchow"
92,Salmonella enterica,BCW_2794,"BCW_2794,""KR 225"",""SRS528772""",Bredeney,PDT000197354.1,2017-03-22T15:10:09Z,Nigeria,chicken,environmental/other,NaN,NaN,NaN,SAMN02368666,GCA_002063755.1,"blaTEM-1=COMPLETE,gyrA_D87N=POINT,mdsA=COMPLET...","antigen_formula=4:l,v:1,7,serotype=Bredeney"
94,Salmonella enterica,BCW_2750,"BCW_2750,""KR167"",""SRS528743""",Onireke,PDT000197377.1,2017-03-22T15:18:10Z,Nigeria,chicken,environmental/other,PDS000001391.75,8.0,14.0,SAMN02368627,GCA_002064225.1,"aadA1=COMPLETE,dfrA1=COMPLETE,gyrA_D87G=POINT,...","antigen_formula=4:b:1,2,serotype=Paratyphi B v..."
95,Salmonella enterica,BCW_2729,"BCW_2729,""KR 84"",""SRS528729""",Mbandaka,PDT000197390.1,2017-03-22T15:21:50Z,Nigeria,chicken,environmental/other,NaN,NaN,NaN,SAMN02368606,GCA_002064495.1,"aadA1=COMPLETE,dfrA15=COMPLETE,gyrA_D87Y=POINT...","antigen_formula=7:z10:e,n,z15,serotype=Mbandaka"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
682,Salmonella enterica,NaN,"D38,""ERS4597431"",""Salmonella""",NaN,PDT001291149.1,2022-04-20T13:38:33Z,Nigeria,Poultry vendoors and farms,NaN,PDS000019581.6,NaN,NaN,SAMEA6869863,GCA_023070735.1,"mdsA=COMPLETE,mdsB=COMPLETE,qnrB19=COMPLETE","antigen_formula=4:z10:1,5,serotype=Ituri"
683,Salmonella enterica,NaN,"D39,""ERS4597432"",""Salmonella""",NaN,PDT001291150.1,2022-04-20T13:38:33Z,Nigeria,Poultry vendoors and farms,NaN,PDS000108531.2,NaN,NaN,SAMEA6869864,GCA_023070715.1,"mdsA=COMPLETE,mdsB=COMPLETE","antigen_formula=7:r:1,2,serotype=Virchow"
684,Salmonella enterica,NaN,"D40,""ERS4597433"",""Salmonella""",NaN,PDT001291151.1,2022-04-20T13:38:33Z,Nigeria,Poultry vendoors and farms,NaN,PDS000108534.1,NaN,NaN,SAMEA6869865,GCA_023070695.1,"fosA7=COMPLETE,fosB=PARTIAL_END_OF_CONTIG,mdsA...","antigen_formula=4:z4,z23:1,2,serotype=Stanleyv..."
685,Salmonella enterica,NaN,"D41,""ERS4597434"",""Salmonella""",NaN,PDT001291152.1,2022-04-20T13:38:34Z,Nigeria,Poultry vendoors and farms,NaN,NaN,NaN,NaN,SAMEA6869866,GCA_023070635.1,"mdsA=COMPLETE,mdsB=COMPLETE","antigen_formula=7:g,s,t:-,serotype=Menston"


In [61]:
# One hot encoding
list(filtered_df["Isolation source"])

['food',
 'Human, faeces',
 'melon seed',
 'Cattle liver',
 'ground chili pepper',
 'ground melon seed egusi',
 'ground melon seed egusi',
 'ground chili pepper',
 'melon seed',
 'melon seed',
 'melon seed',
 'tissue (Gallus gallus)',
 'tissue (Gallus gallus)',
 'tissue (Gallus gallus)',
 'tissue (Gallus gallus)',
 'tissue (Gallus gallus)',
 'tissue (Gallus gallus)',
 'tissue (Gallus gallus)',
 'tissue (Gallus gallus)',
 'tissue (Gallus gallus)',
 'tissue (Gallus gallus)',
 'tissue (Gallus gallus)',
 'tissue (Gallus gallus)',
 'tissue (Gallus gallus)',
 'tissue (Gallus gallus)',
 'tissue (Gallus gallus)',
 'rectal swab (Bos taurus)',
 'tissue (Gallus gallus)',
 'tissue (Gallus gallus)',
 'tissue (Gallus gallus)',
 'tissue (Gallus gallus)',
 'tissue (Gallus gallus)',
 'tissue (Gallus gallus)',
 'tissue (Gallus gallus)',
 'tissue (Gallus gallus)',
 'tissue (Gallus gallus)',
 'tissue (Gallus gallus)',
 'tissue (Gallus gallus)',
 'cloacal swab (Gallus gallus)',
 'tissue (Gallus gallus)',
 

In [37]:
# Replicate for trial 


# Create a new dataset with 500 rows
# Replicate the data 5 times (total 500 rows)
for _ in range(6):
    replicated_data = pd.concat([df, df], ignore_index=True)


In [38]:
replicated_data

,Biosample,sra_accession
0,SAMN20456381,SRR15403520
1,SAMN20456395,SRR15369306
2,SAMN20456385,SRR15369305
3,SAMN20456390,SRR15369303
4,SAMN20456392,SRR15369302
...,...,...
263,SAMN03576959,SRR2163255
264,SAMN03576956,SRR2163254
265,SAMN03576963,SRR2163247
266,SAMN03576962,SRR2163246


In [39]:
# Step 2: Remove constant columns
non_constant_cols = replicated_data.columns[replicated_data.nunique() > 1]
replicated_data = replicated_data[non_constant_cols]
replicated_data.reindex()

,Biosample,sra_accession
0,SAMN20456381,SRR15403520
1,SAMN20456395,SRR15369306
2,SAMN20456385,SRR15369305
3,SAMN20456390,SRR15369303
4,SAMN20456392,SRR15369302
...,...,...
263,SAMN03576959,SRR2163255
264,SAMN03576956,SRR2163254
265,SAMN03576963,SRR2163247
266,SAMN03576962,SRR2163246


In [40]:
# Step 2: Perform one-hot encoding for all categorical columns
replicated_data_encoded = pd.get_dummies(df)
replicated_data_encoded

,Biosample_SAMN02843543,Biosample_SAMN02844775,Biosample_SAMN03576951,Biosample_SAMN03576952,Biosample_SAMN03576953,Biosample_SAMN03576954,Biosample_SAMN03576955,Biosample_SAMN03576956,Biosample_SAMN03576958,Biosample_SAMN03576959,...,sra_accession_SRR3747672,sra_accession_SRR3924533,sra_accession_SRR3924536,sra_accession_SRR4123985,sra_accession_SRR4123986,sra_accession_SRR4243040,sra_accession_SRR4733853,sra_accession_SRR4733857,sra_accession_SRR5972858,sra_accession_SRR6958628
0,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
1,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
2,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
3,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
4,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
129,False,False,False,False,False,False,False,False,False,True,...,False,False,False,False,False,False,False,False,False,False
130,False,False,False,False,False,False,False,True,False,False,...,False,False,False,False,False,False,False,False,False,False
131,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
132,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False


In [117]:
# Step 3: Scale the data
# from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()  # or StandardScaler()
scaled_data = scaler.fit_transform(replicated_data)

# Convert the scaled numpy array back to a DataFrame with the original columns
scaled_data_df = pd.DataFrame(scaled_data, columns=non_constant_cols)
scaled_data_df

NameError: name 'MinMaxScaler' is not defined

In [4]:
from sklearn.preprocessing import MinMaxScaler
